In [1]:
# Use autoplot 207's code for this app
import sys
sys.path.insert(0, "/opt/iem/htdocs/plotting/auto/scripts200/")
from p207 import load_data, compute_grid_bounds, add_zeros, do_analysis, USEME
from pyiem.plot import MapPlot, nwssnow
from pyiem.util import get_dbconn, utc
import numpy as np

In [5]:
STORM_NUMBER = 21
TITLE = "12-13 February 2021"
SUBTITLE = "8 AM 13 February 2021"
sts = utc(2021, 2, 12, 20)
ets = utc(2021, 2, 14, 17, 0)
# Get available data
ctx = {'coop': 'yes', 'csector': 'IA', 't': 'state', 'sz': 30, 'z': 'yes', 'f': 'linear', 'v': 'snow'}
df = load_data(ctx, sts, ets)
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx)
# add zeros and QC
df = add_zeros(df, ctx)

In [3]:
def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - IEM Snowfall Total Analysis" % (TITLE, ),
        subtitle=("Snowfall totals till %s from NWS COOP, LSR, CoCoRaHS Reports; "
                  "IEM 2020-2021 Winter Storm #%s") % (SUBTITLE, STORM_NUMBER),
        twitter=True,
    )
    mp.contourf(
        lons, lats, vals, np.array([0.1, 1, 2, 3, 4, 6, 8, 12, 18, 24, 30, 36]), cmap=nwssnow(), clip_on=True)
    df_useme_plot = df2[(df2['val']>=lower)&(df2['val'] < upper)]
    print(df_useme_plot)
    mp.drawcounties()
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(df_useme_plot['lon'], df_useme_plot['lat'], df_useme_plot['val'],
            '%s', labels=df_useme_plot['nwsli'].values,
            textsize=10, labeltextsize=10, labelbuffer=1)
    return mp

In [14]:
cull = [831, 1180, 705, 846]
if cull:
    df.loc[df['nwsli'].isin(cull), USEME] = False
# df.loc[df[df["val"] < 1].index, USEME] = False
hardcode = [
    ['IA-BH-17', 2],
    ['CLUI4', 2],
]
for nwsli, val in hardcode:
    df.loc[df['nwsli'] == nwsli, 'val'] = val

ctx['csector'] = "IA"
mp = workflow(ctx, df, isfinal=True, lower=3, upper=7)
res = mp.postprocess(filename='210214.png')
mp.close()

     state  wfo  val         lon        lat                              geo  \
378     NE  CYS  6.7 -103.650000  41.820000  POINT (-302458.187 -346834.354)   
379     CO  GJT  6.7 -107.840000  38.480000  POINT (-682303.070 -692481.522)   
380     WA  SEW  6.6 -123.070000  46.830000  POINT (-1725013.971 454416.491)   
381     CO  GJT  6.6 -106.760000  37.040000  POINT (-600648.703 -860188.217)   
382     WA  SEW  6.5 -122.020000  47.750000  POINT (-1620652.570 531406.571)   
...    ...  ...  ...         ...        ...                              ...   
1834    WY  RIW  3.0 -107.974400  41.671400  POINT (-661335.707 -337949.811)   
1835    CO  BOU  3.0 -106.035300  39.626100  POINT (-516895.834 -578515.456)   
1836    IA  DMX  3.0  -92.454300  40.626167   POINT (636176.189 -457131.922)   
1837    VA  AKQ  3.0  -75.821700  37.583100  POINT (2098279.556 -516723.059)   
1838    IA  DMX  3.0  -93.651113  42.107890   POINT (523259.496 -301292.101)   

      used_for_analysis     nwsli  plot